In [ ]:
import subprocess
import random
import cv2
import os
# Đường dẫn đến video

for i in range(1, 10+1):
    video_path = f'../data/sample_tv360/video_chunk/chunk_{i}.mp4'

    # Tính tổng thời gian của video.
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration =  81 
    print("Đây là duration", duration)
    cap.release()

    # Chọn thời gian bắt đầu ngẫu nhiên (bảo đảm có đủ 25 giây)
    start_time = random.uniform(0, max(0, duration - 32))
    start_time_formatted = f"{int(start_time // 3600):02}:{int((start_time % 3600) // 60):02}:{int(start_time % 60):02}"
    print(start_time_formatted)
    # Đường dẫn đến video đầu ra
    if not os.path.exists('../data/sample_tv360/gt'):
        os.makedirs('../data/sample_tv360/gt')
    output_path = f'../data/sample_tv360/gt/chunk_based_audo_{i}_gt.mp4'

    # Lệnh ffmpeg để trích xuất video
    ffmpeg_cmd = [
        'ffmpeg',
        '-y',
        '-ss', start_time_formatted,  # Thời gian bắt đầu
        '-i', video_path,             # Video đầu vào
        '-to', '32',                   # Thời lượng clip chính xác là 27 giây
        '-strict', 'experimental',    # Sử dụng codec âm thanh aac với tùy chọn strict
        '-shortest',                  # Đảm bảo rằng video sẽ kết thúc đúng khi đạt thời gian yêu cầu
        '-r', '30',
        output_path                   # Đường dẫn đầu ra
    ]

    # Thực thi lệnh ffmpeg
    subprocess.run(ffmpeg_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print(f"Video được lưu mỗi {duration/3}s: {output_path}")


In [ ]:
import os
import subprocess
from tqdm import tqdm

# Đường dẫn đến thư mục chứa các video cần nối
final_dir = '../data/sample_tv360/gt'

# Tạo thư mục output nếu chưa tồn tại
final_output_dir = '../data/sample_tv360/final'
if not os.path.exists(final_output_dir):
    os.makedirs(final_output_dir)

# Đường dẫn đến video sau khi nối
output_video = os.path.join(final_output_dir, 'complete_video_gt.mp4')
file_list_path = os.path.join(final_output_dir, 'merges_gt.txt')

# Tạo file danh sách các video để nối với tqdm
with open(file_list_path, 'w') as f:
    for video_file in sorted(os.listdir(final_dir), key=lambda x: int(x.split('_')[1])):
        file_path = os.path.join(final_dir, video_file)
        # Thêm đường dẫn chính xác vào file_list_path
        f.write(f"file '{os.path.abspath(file_path)}'\n")

# Kiểm tra nội dung file_list.txt
print("\nList of files to be concatenated:")
with open(file_list_path, 'r') as f:
    print(f.read())

# Lệnh ffmpeg để nối video, chuyển hướng stdout và stderr để ẩn thông báo từ ffmpeg
concat_cmd = [
    'ffmpeg',
    '-y',
    '-f', 'concat',
    '-safe', '0',
    '-i', file_list_path,
    '-c:v', 'libopenh264',  # Mã hóa lại video với libopenh264
    '-c:a', 'aac',          # Mã hóa lại audio với aac
    '-r', '30',
    '-strict', 'experimental',
    output_video
]

# Chạy lệnh nối video và tắt đầu ra của ffmpeg
print("Running ffmpeg to concatenate videos...")
subprocess.run(concat_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Kiểm tra xem file output có được tạo thành công không
if os.path.exists(output_video):
    print(f"\nAll videos have been successfully concatenated into {output_video}")
else:
    print("\nError: The concatenation process failed. Please check the ffmpeg output for details.")
